In [42]:
import pandas as pd
from functools import reduce
import matplotlib.pyplot as plt

In [43]:
file_path = '../data/nba_points_2024_2025.xlsx'
df = pd.read_excel(file_path, usecols="A:T")

In [44]:
df.columns

Index(['Date', 'Weekday', 'Away', 'Home', 'NBA Cup Game', 'Crew Chief',
       'Referee', 'Umpire', 'Open Total', 'Open Home Spread', 'Away Points',
       'Home Points', 'Actual Total', 'Actual Home Spread', 'Game Count',
       'Over', 'Over Amount', 'Under', 'Under Amount', 'Push'],
      dtype='object')

In [45]:
df['Home Win'] = (df['Home Points'] > df['Away Points']).astype(int)
df['Away Win'] = (df['Away Points'] > df['Home Points']).astype(int)

In [46]:
print('GAMES TRACKED: ', round(df['Game Count'].sum()))
print('AVG. OPEN TOTAL: ', round(df['Open Total'].mean(), 2))
print('AVG. ACTUAL TOTAL: ', round(df['Actual Total'].mean(), 2))
print('OVERS: ', round(df['Over'].sum()/df['Game Count'].sum() * 100, 2), '%')
print('UNDERS: ', round(df['Under'].sum()/df['Game Count'].sum() * 100, 2), '%')

GAMES TRACKED:  559
AVG. OPEN TOTAL:  225.69
AVG. ACTUAL TOTAL:  225.64
OVERS:  50.09 %
UNDERS:  49.73 %


In [47]:
game_counts = df.groupby(['Crew Chief'])['Game Count'].sum().reset_index(name='games')
over_percs = round(df.groupby(['Crew Chief'])['Over'].sum()/df.groupby('Crew Chief')['Game Count'].sum() * 100, 2).reset_index(name='over_percentage')
over_by_avg = round(df.groupby(['Crew Chief'])['Over Amount'].mean(), 2).reset_index(name='over_by_avg')
over_by_std = round(df.groupby(['Crew Chief'])['Over Amount'].std(), 2).reset_index(name='over_by_std')
under_by_avg = round(df.groupby(['Crew Chief'])['Under Amount'].mean(), 2).reset_index(name='under_by_avg')
under_by_std = round(df.groupby(['Crew Chief'])['Under Amount'].std(), 2).reset_index(name='under_by_std')
under_percs = round(df.groupby(['Crew Chief'])['Under'].sum()/df.groupby('Crew Chief')['Game Count'].sum() * 100, 2).reset_index(name='under_percentage')
avg_totals = round(df.groupby(['Crew Chief'])['Actual Total'].mean(), 2).reset_index(name='mean_actual_total')
std_totals = round(df.groupby(['Crew Chief'])['Actual Total'].std(), 2).reset_index(name='std_actual_total')
avg_opening_total = round(df.groupby(['Crew Chief'])['Open Total'].mean(), 2).reset_index(name='mean_open_total')

groupby_results = [game_counts, over_percs, over_by_avg, over_by_std, under_by_avg, under_by_std, under_percs, avg_totals, std_totals, avg_opening_total]
chief_stats = reduce(lambda left, right: pd.merge(left, right, on='Crew Chief'), groupby_results)

In [48]:
chief_stats[(chief_stats.games >= 15) &
            ((chief_stats.over_percentage > 66) | (chief_stats.under_percentage > 66))  ][['Crew Chief', 'games', 'over_percentage', 'under_percentage']].sort_values(by='over_percentage')

,Crew Chief,games,over_percentage,under_percentage
7,Ed Malloy,29.0,24.14,75.86
22,Pat Fraher,19.0,31.58,68.42
24,Scott Foster,24.0,70.83,29.17


In [49]:
home_df = df[['Home', 'Game Count', 'Over', 'Under', 'Home Points', 'Home Win']].copy()
home_df['Location'] = 'Home'
home_df.rename(columns={'Home':'Team', 'Home Win':'Win', 'Home Points':'Points'}, inplace=True)

away_df = df[['Away', 'Game Count', 'Over', 'Under', 'Away Points', 'Away Win']].copy()
away_df['Location'] = 'Away'
away_df.rename(columns={'Away':'Team', 'Away Win':'Win', 'Away Points':'Points'}, inplace=True)

home_away_df = pd.concat([home_df, away_df])

In [50]:
round(home_away_df.groupby(['Team'])['Over'].sum()/
                       home_away_df.groupby(['Team'])['Game Count'].sum() * 100, 2).reset_index(name='Over Percentage').sort_values(by='Over Percentage', ascending=False)[0:5]

,Team,Over Percentage
16,MIA,100.00
14,MEM,72.97
7,DEN,64.86
0,ATL,63.16
5,CLE,59.46


In [51]:
round(home_away_df.groupby(['Team'])['Under'].sum()/
                       home_away_df.groupby(['Team'])['Game Count'].sum() * 100, 2).reset_index(name='Under Percentage').sort_values(by='Under Percentage', ascending=False)[0:5]

,Team,Under Percentage
3,CHA,65.71
9,GSW,63.16
27,SAS,62.16
12,LAC,62.16
2,BOS,57.89
